Задача 2:
При покупке авиабилета пользователям предлагаются дополнительные услуги (страховка, онлайн-регистрация, выбор места, и т.д.). Для каждого заказа определите вероятность покупки каждой дополнительной услуги. В качестве сабмита принимается .csv файл с шестью колонками - id заказа (orderid) и вероятность покупки пяти дополнительных услуг.

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
import lightgbm as lgb
import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn')
%matplotlib inline

from IPython.core.display import display, HTML
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
display(HTML("<style>.container { width:95% !important; }</style>"))
import datetime

In [ ]:
train = pd.read_csv('onetwotrip_challenge_train.csv')
test = pd.read_csv('onetwotrip_challenge_test.csv')

le = preprocessing.LabelEncoder()
le.fit(pd.concat([train['userid'],test['userid']]))
train['userid'] = le.transform(train['userid'])
test['userid'] = le.transform(test['userid'])

train['COUNT_id'] = train['userid'].map(train['userid'].value_counts())
test['COUNT_id'] = test['userid'].map(test['userid'].value_counts())

fichi = [c for c in train.columns if c not in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]

In [ ]:
unique14 = np.unique(pd.concat([train['field14'],test['field14']]))
new_unique14 = np.round(unique14 / 0.14006639 - 0.27860731).astype(int)
dict14 = {unique14[i]: new_unique14[i] for i in range(len(unique14))}
train['field14'] = train['field14'].apply(lambda x: dict14[x])
test['field14'] = test['field14'].apply(lambda x: dict14[x])

train['field14'] = train['field14'] + np.absolute(pd.concat([train['field14'],test['field14']]).min())
test['field14'] = test['field14'] + np.absolute(pd.concat([train['field14'],test['field14']]).min())

unique1 = np.unique(pd.concat([train['field1'],test['field1']]))
new_unique1 = np.round(unique1  / 0.077571 + 0.0765905).astype(int)
dict1 = {unique1[i]: new_unique1[i] for i in range(len(unique1))}
train['field1'] = train['field1'].apply(lambda x: dict1[x])
test['field1'] = test['field1'].apply(lambda x: dict1[x])        
        
train['field1'] = train['field1'] + np.absolute(pd.concat([train['field1'],test['field1']]).min())
test['field1'] = test['field1'] + np.absolute(pd.concat([train['field1'],test['field1']]).min())

train['ticket_price'] = train['field1'] / train['field15']
test['ticket_price'] = test['field1'] / test['field15']

In [ ]:
#temp
features = [c for c in train.columns if c not in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]
for f in features:
    train['COUNT_{}'.format(f)] = train[f].map(pd.concat([train[f],test[f]]).value_counts())
    test['COUNT_{}'.format(f)] = test[f].map(pd.concat([train[f],test[f]]).value_counts())
for df in [train,test]:
    for f in features:
        df['{}_STD'.format(f)] = df['userid'].map(pd.concat([train[['userid',f]],test[['userid',f]]]).groupby('userid')[f].std())
        df['{}_MEAN'.format(f)] = df['userid'].map(pd.concat([train[['userid',f]],test[['userid',f]]]).groupby('userid')[f].mean())
        df['{}_MAX'.format(f)] = df['userid'].map(pd.concat([train[['userid',f]],test[['userid',f]]]).groupby('userid')[f].max())
        df['{}_MIN'.format(f)] = df['userid'].map(pd.concat([train[['userid',f]],test[['userid',f]]]).groupby('userid')[f].min())

In [ ]:
print(train['goal21'].value_counts()),
print(train['goal22'].value_counts()),
print(train['goal23'].value_counts()),
print(train['goal24'].value_counts()),
print(train['goal25'].value_counts())

In [ ]:
for f in ['field12','field16']:
    train['COUNT_{}'.format(f)] = train[f].map(pd.concat([train[f],test[f]]).value_counts())
    test['COUNT_{}'.format(f)] = test[f].map(pd.concat([train[f],test[f]]).value_counts())
    
col = 'userid'
for df in [train,test]:
    for f in ['field26']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
        df['{}_{}_MEAN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].mean())
        df['{}_{}_MAX'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].max())
        
train['ticket_price'] = train['field1'] / train['field15']
test['ticket_price'] = test['field1'] / test['field15']
for df in [train,test]:
    for f in ['ticket_price']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
del train['ticket_price'], test['ticket_price']
for df in [train,test]:
    for f in ['field0']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
for df in [train,test]:
    for f in ['field13']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())
for df in [train,test]:
    for f in ['field1']:
        df['{}_{}_MEAN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].mean())
for df in [train,test]:
    for f in ['field16']:
        df['{}_{}_MEAN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].mean())
        df['{}_{}_MIN'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].min())
for df in [train,test]:
    for f in ['field12']:
        df['{}_{}_STD'.format(col,f)] = df[col].map(pd.concat([train[[col,f]],test[[col,f]]]).groupby(col)[f].std())  

In [ ]:
for df in [train,test]:
    df.loc[df['field21']==2,'field2'] = df['field2'] + 12
    df.loc[df['field21']==2,'field3'] = df['field3'] + 12

In [ ]:
train.to_pickle('tr')
test.to_pickle('te')

In [ ]:
features = [c for c in train.columns if c not in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]

subfor5auc = pd.read_csv('onetwotrip_challenge_sub2.csv')
subfor5auc = subfor5auc[subfor5auc.columns[1:]]

for r in [106,148,170]:

    y_pred = np.zeros(test[features].shape[0])

    for col in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25']:
        print(col,r)
        train = pd.read_pickle('tr')
        test = pd.read_pickle('te')

        features = [c for c in train.columns if c not in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1','orderid','userid']]

        train = train.sort_values('goal1').reset_index(drop=True)
        vc = train['goal1'].value_counts()
        vc = dict(sorted(vc.items()))
        df = pd.DataFrame()
        train['indexcol'],i = 0,1
        for k,v in vc.items():
            step = train.shape[0]/v
            indent = train.shape[0]/(v+1)
            df2 = train[train['goal1'] == k].sample(v,random_state = r).reset_index(drop=True)
            for j in range(0, v):
                df2.at[j, 'indexcol'] = indent + j*step + 0.000001*i
            df = pd.concat([df2,df])
            i+=1
        train = df.sort_values('indexcol', ascending=False).reset_index(drop=True)
        del train['indexcol'], df, df2, vc
        #for i in range(5): print(i+1,train['goal1'][i*39211:39211*(i+1)].value_counts()[1])

        N=5
        folds = KFold(n_splits=N)
        target = train[col].copy()

        lgb_param = {
            'min_data_in_leaf': 16,
            'num_leaves': 128, 
            'learning_rate': 0.005,
            'min_child_weight': 0.01,
            'bagging_fraction': 0.85, 
            'feature_fraction': 0.85,
            'max_depth': -1,
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'verbose': 1,
            'metric':'auc',
        }

        importance = pd.DataFrame(np.zeros((train[features].shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train[features].columns)
        scores = []
        oof = np.zeros(train[features].shape[0])

        for fold, (trn_idx, val_idx) in enumerate(folds.split(train[features], target), 1):
            #print('Fold {}'.format(fold))    
            trn_data = lgb.Dataset(train[features].iloc[trn_idx, :].values, label=target.iloc[trn_idx].values)
            val_data = lgb.Dataset(train[features].iloc[val_idx, :].values, label=target.iloc[val_idx].values)   
            clf = lgb.train(lgb_param, trn_data, 800, valid_sets=[trn_data, val_data], verbose_eval=1001, early_stopping_rounds=1001)
            predictions = clf.predict(train[features].iloc[val_idx, :].values) 
            importance.iloc[:, fold - 1] = clf.feature_importance()
            oof[val_idx] = predictions
            score = roc_auc_score(target.iloc[val_idx].values, predictions)
            scores.append(score)
            print('Fold {} ROC AUC Score:'.format(fold, score))
            y_pred += clf.predict(test[features]) / N*3
            del trn_data, val_data, predictions
            gc.collect()
        print(r, '######### Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))
        gc.collect()
    subfor5auc[col] = y_pred

In [ ]:
probas.to_csv('sub2.csv')